In [0]:
import requests
import json

API_KEY = "test_13d0a343623562abf5bf8323da103d"
CHAMPIONSHIP_ID = 10
URL = f"https://api.api-futebol.com.br/v1/campeonatos/{CHAMPIONSHIP_ID}/partidas"
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(URL, headers=headers)
print("Status Code:", response.status_code)

# Tenta carregar JSON
data_json = response.json()

# Exibe a resposta completa de forma legível
print(json.dumps(data_json, indent=4))


In [0]:
from pyspark.sql import SparkSession, Row
from datetime import datetime

spark = SparkSession.builder.getOrCreate()

# ----------------------
# CONFIGURAÇÃO
# ----------------------
delta_table = "brasileirao.raw.tb_partidas_raw"

# Exemplo de dados raspados (substitua pelo seu DataFrame real)
partidas = [
    Row(Mandante="ASA", Visitante="Atlético-GO", Data="2025-02-19", Hora="21:30", Placar="1 x 0"),
    Row(Mandante="Jequié", Visitante="Retrô", Data="2025-02-26", Hora="19:30", Placar="2 x 2")
]

df = spark.createDataFrame(partidas)

# ----------------------
# VERIFICAR SE A TABELA EXISTE (SQL)
# ----------------------
tabela_existe = spark.sql(f"SHOW TABLES IN brasileirao LIKE 'tb_partidas_raw'").count() > 0

if tabela_existe:
    print(f"Tabela {delta_table} existe. Apagando...")
    spark.sql(f"DROP TABLE brasileirao.raw.tb_partidas_raw")

# ----------------------
# SALVAR NOVA TABELA DELTA
# ----------------------
df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(delta_table)

print(f"Delta Table criada com sucesso: {delta_table}")

# ----------------------
# MOSTRAR DADOS
# ----------------------
df.show(truncate=False)


In [0]:
# ----------------------
# IMPORTS
# ----------------------
import requests
from pyspark.sql import SparkSession, Row
from datetime import datetime
import json

# ----------------------
# CONFIGURAÇÃO
# ----------------------
API_KEY = "test_13d0a343623562abf5bf8323da103d"
CHAMPIONSHIP_ID = 10
URL = f"https://api.api-futebol.com.br/v1/campeonatos/{CHAMPIONSHIP_ID}/partidas"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

DELTA_TABLE = "brasileirao.raw.tb_partidas_raw"

# ----------------------
# INICIA SPARK
# ----------------------
spark = SparkSession.builder.getOrCreate()

# ----------------------
# CONSULTA API
# ----------------------
response = requests.get(URL, headers=HEADERS)
if response.status_code != 200:
    raise Exception(f"Erro na API: {response.status_code} - {response.text}")

data_json = response.json()

# ----------------------
# TRATAR DADOS
# ----------------------
partidas = []

# A API retorna um dicionário complexo: percorre "primeira-fase" > "chave-X" > "ida"
fase = data_json.get("partidas", {}).get("partidas", {}).get("primeira-fase", {})

for chave, info_chave in fase.items():
    ida = info_chave.get("ida", {})
    if ida:  # se existir ida
        mandante = ida.get("time_mandante", {}).get("nome_popular", "")
        visitante = ida.get("time_visitante", {}).get("nome_popular", "")
        data_real = ida.get("data_realizacao", "")
        hora_real = ida.get("hora_realizacao", "")
        placar_home = ida.get("placar_home", ida.get("placar", {}).get("home", 0))
        placar_away = ida.get("placar_away", ida.get("placar", {}).get("away", 0))
        placar = f"{placar_home} x {placar_away}" if isinstance(placar_home, int) else ida.get("placar", "0 x 0")
        
        partidas.append(
            Row(
                Mandante=mandante,
                Visitante=visitante,
                Data=data_real,
                Hora=hora_real,
                Placar=placar
            )
        )

# ----------------------
# CRIA DATAFRAME
# ----------------------
df = spark.createDataFrame(partidas)

# ----------------------
# VERIFICA TABELA EXISTENTE E REMOVE
# ----------------------
tabela_existe = spark.sql(f"SHOW TABLES IN brasileirao LIKE 'tb_partidas_raw'").count() > 0

if tabela_existe:
    print(f"Tabela {DELTA_TABLE} existe. Apagando...")
    spark.sql(f"DROP TABLE {DELTA_TABLE}")

# ----------------------
# CRIA NOVA DELTA TABLE
# ----------------------
df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(DELTA_TABLE)

print(f"Delta Table criada com sucesso: {DELTA_TABLE}")

# ----------------------
# MOSTRAR DADOS
# ----------------------
df.show(truncate=False)


In [0]:
import requests
from pyspark.sql import SparkSession, Row
from datetime import datetime

# ----------------------
# CONFIGURAÇÃO
# ----------------------
API_KEY = "test_13d0a343623562abf5bf8323da103d"
CHAMPIONSHIP_ID = 10
URL = f"https://api.api-futebol.com.br/v1/campeonatos/{CHAMPIONSHIP_ID}/partidas"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
DELTA_TABLE = "brasileirao.raw.tb_partidas_raw"

spark = SparkSession.builder.getOrCreate()

# ----------------------
# REQUISIÇÃO DAS PARTIDAS
# ----------------------
response = requests.get(URL, headers=HEADERS)
if response.status_code != 200:
    raise Exception(f"Erro na API: {response.status_code} - {response.text}")

data_json = response.json()

# ----------------------
# EXTRAÇÃO DE DADOS
# ----------------------
partidas = []

# A API tem "partidas" → fases → chaves → ida/volta
partidas_json = data_json.get("partidas", {})

for fase_key, fase_val in partidas_json.items():
    for chave_key, chave_val in fase_val.items():
        for tipo in ["ida", "volta"]:
            detalhe = chave_val.get(tipo)
            if detalhe is None:
                continue

            # Extrair dados da partida
            mandante = detalhe.get("time_mandante", {}).get("nome_popular", "")
            visitante = detalhe.get("time_visitante", {}).get("nome_popular", "")
            data_jogo = detalhe.get("data_realizacao", "")
            hora_jogo = detalhe.get("hora_realizacao", "")
            status = detalhe.get("status", "")
            slug = detalhe.get("slug", "")
            link = detalhe.get("_link", "")
            rodada = detalhe.get("rodada", "")
            temporada = detalhe.get("temporada", "")
            estadio = detalhe.get("estadio", "")

            # Placar
            placar_home = detalhe.get("placar_home", "")
            placar_away = detalhe.get("placar_away", "")
            placar = f"{placar_home} x {placar_away}" if placar_home or placar_away else ""

            # Gols (quem fez, minuto, time)
            gols = []
            for gol in detalhe.get("gols", []):
                minuto = gol.get("minuto", "")
                autor = gol.get("autor", "")
                time = gol.get("time", "")
                gols.append(f"{minuto}' {autor} ({time})")
            gols_str = "; ".join(gols)

            partidas.append(Row(
                Mandante=mandante,
                Visitante=visitante,
                Data=data_jogo,
                Hora=hora_jogo,
                Placar=placar,
                Estadio=estadio,
                Rodada=rodada,
                Status=status,
                Temporada=temporada,
                Slug=slug,
                Link=link,
                Gols=gols_str,
                Data_Extracao=datetime.now()
            ))

# ----------------------
# CHECAR SE HÁ PARTIDAS
# ----------------------
if not partidas:
    print("Nenhuma partida encontrada na API.")
else:
    df = spark.createDataFrame(partidas)

    # ----------------------
    # VERIFICAR SE TABELA EXISTE
    # ----------------------
    tabela_existe = spark.sql(f"SHOW TABLES IN brasileirao LIKE 'tb_partidas_raw'").count() > 0

    if tabela_existe:
        print(f"Tabela {DELTA_TABLE} existe. Apagando...")
        spark.sql(f"DROP TABLE {DELTA_TABLE}")

    # ----------------------
    # SALVAR NOVA DELTA TABLE
    # ----------------------
    df.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(DELTA_TABLE)

    print(f"Delta Table criada com sucesso: {DELTA_TABLE}")

    # ----------------------
    # MOSTRAR DADOS
    # ----------------------
    df.show(truncate=False)
